In [6]:
import pandas as pd
import numpy as np

# 1. Cargar los datos
df1 = pd.read_csv('C:/Users/Hernán Ifrán/Downloads/df_prediccion_all.csv')

productosPredecir = df = pd.read_csv('C:/Users/Hernán Ifrán/Downloads/productos_a_predecir.txt', sep='\t')
tb_productos = pd.read_csv('C:/Users/Hernán Ifrán/Downloads/tb_productos.txt', sep='\t')
tb_productos_descripcion = pd.read_csv('C:/Users/Hernán Ifrán/Downloads/tb_productos_descripcion.txt', sep='\t')



In [8]:
print(df1.columns)


Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion', 'stock_final'],
      dtype='object')


In [11]:
## Convertir el periodo a formato datetime
df1['periodo'] = pd.to_datetime(df1['periodo'], format='%Y%m')

# Agregar los datos por periodo y product_id para obtener la serie temporal
ts = df1.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Asegurarse de que las columnas tengan el mismo tipo y formato
ts['product_id'] = ts['product_id'].astype(int)
ts['periodo'] = pd.to_datetime(ts['periodo'])
# Unir las categorías de productos desde el archivo tb_productos
ts = ts.merge(tb_productos[['product_id', 'cat1', 'cat2', 'cat3', 'brand']], on='product_id', how='left')

In [12]:
print(ts.columns)

Index(['periodo', 'product_id', 'tn', 'cat1', 'cat2', 'cat3', 'brand'], dtype='object')


In [14]:
import h2o
from h2o.automl import H2OAutoML



In [18]:
h2o.init()
h2o_ts = h2o.H2OFrame(ts)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,4 mins 32 secs
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,7 months and 19 days
H2O_cluster_name:,H2O_from_python_Hernan_Ifran_ckbz99
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.514 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [20]:
# Especificar las columnas 'x' y la columna objetivo 'y'
x = h2o_ts.columns  # Todas las columnas excepto la objetivo
y = 'tn'  # Nombre de la columna objetivo

# Configurar y ejecutar AutoML
aml = H2OAutoML(max_models=20, seed=1)  # Puedes ajustar max_models según tus necesidades
aml.train(x=x, y=y, training_frame=h2o_ts)

# Ver los detalles del mejor modelo
lb = aml.leaderboard
print(lb.head())  # Muestra los mejores modelos


AutoML progress: |█
14:12:57.677: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
model_id                                                    rmse       mse       mae    rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20240627_141257     28.0268   785.5     9.71443      nan                   785.5
GBM_4_AutoML_1_20240627_141257                           28.4392   808.79   10.2912       nan                   808.79
StackedEnsemble_BestOfFamily_1_AutoML_1_20240627_141257  28.484    811.336  10.3295       nan                   811.336
GBM_grid_1_AutoML_1_20240627_141257_model_5              28.5148   813.095   9.58454      nan                   813.095
GBM_3_AutoML_1_20240627_141257                           28.9467   837.911  10.6237       nan                   837.911
GBM_2_AutoML_1_20240627_141257                           29.3177   859.53   10.8063       nan                   859.53
GBM_5_

In [22]:
# Iniciar H2O (si aún no lo has hecho)
h2o.init()

# Suponiendo que ya has ejecutado AutoML y tienes el objeto aml
# Obtener el modelo ensamblado específico del leaderboard
ensemble_model_id = 'StackedEnsemble_AllModels_1_AutoML_1_20240627_141257'
ensemble_model = h2o.get_model(ensemble_model_id)

# Ver los modelos base que se usaron en el ensamblado apilado
base_models = ensemble_model.base_models

print("Base models used in the Stacked Ensemble:")
for base_model_id in base_models:
    print(base_model_id)

# Opcional: obtener más detalles de cada modelo base
for base_model_id in base_models:
    base_model = h2o.get_model(base_model_id)
    print(base_model)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,2 hours 5 mins
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,7 months and 19 days
H2O_cluster_name:,H2O_from_python_Hernan_Ifran_ckbz99
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.225 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Base models used in the Stacked Ensemble:
GBM_4_AutoML_1_20240627_141257
GBM_grid_1_AutoML_1_20240627_141257_model_5
GBM_3_AutoML_1_20240627_141257
GBM_2_AutoML_1_20240627_141257
GBM_5_AutoML_1_20240627_141257
GBM_grid_1_AutoML_1_20240627_141257_model_2
GBM_grid_1_AutoML_1_20240627_141257_model_3
GBM_grid_1_AutoML_1_20240627_141257_model_1
GBM_1_AutoML_1_20240627_141257
GBM_grid_1_AutoML_1_20240627_141257_model_4
XRT_1_AutoML_1_20240627_141257
DRF_1_AutoML_1_20240627_141257
DeepLearning_grid_1_AutoML_1_20240627_141257_model_2
DeepLearning_grid_3_AutoML_1_20240627_141257_model_1
DeepLearning_grid_1_AutoML_1_20240627_141257_model_1
DeepLearning_grid_2_AutoML_1_20240627_141257_model_1
DeepLearning_grid_2_AutoML_1_20240627_141257_model_2
DeepLearning_1_AutoML_1_20240627_141257
DeepLearning_grid_3_AutoML_1_20240627_141257_model_2
GLM_1_AutoML_1_20240627_141257
Model Details
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_1_20240627_141257


Model Summary: 
 